In [1]:
!pip install -q sumeval==0.2.2
!pip install -q nlpaug==1.1.3
!pip install -q simpletransformers==0.60.9

In [2]:
import gc
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split

import nlpaug.augmenter.word as naw
from sumeval.metrics.rouge import RougeCalculator

import torch
from simpletransformers.t5 import T5Model, T5Args

print('Pytorch version: %s'  % torch.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Pytorch version: 2.0.0


In [3]:
warnings.simplefilter('ignore')
pd.set_option('display.max_colwidth', 10000)
cuda =  torch.cuda.is_available()

In [4]:
cuda

True

In [5]:
df_train = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv', encoding='ISO-8859-1').dropna().reset_index(drop=True)
df_val = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv', encoding='ISO-8859-1').dropna().reset_index(drop=True)
df_test = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv', encoding='ISO-8859-1').dropna().reset_index(drop=True)

In [6]:
df_train = df_train.drop(['id'], axis=1)
df_val = df_val.drop(['id'], axis=1)
df_test = df_test.drop(['id'], axis=1)

In [7]:
df_train = df_train.rename(columns={'article': 'input_text', 'highlights': 'target_text'}).reindex(columns=['input_text', 'target_text'])
df_train['prefix'] = ''

df_test =df_test.rename(columns={'article': 'input_text', 'highlights': 'target_text'}).reindex(columns=['input_text', 'target_text'])
df_test['prefix'] = ''

df_val =df_val.rename(columns={'article': 'input_text', 'highlights': 'target_text'}).reindex(columns=['input_text', 'target_text'])
df_val['prefix'] = ''

In [8]:
len(df_train)

287113

In [9]:
len(df_test)

11490

In [10]:
len(df_val)

13368

In [11]:
df_train.head(5)

input_text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five chu

In [12]:
df_test.head(5)

input_text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers, crashing elbows and seat back kicking? Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased . Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches . Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News. The distance between two seats from one point on a seat to the same point on the seat behind it is known as the

In [13]:
df_val.head(5)

,input_text,target_text,prefix
0,"Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDBÂ page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show, her iMDB page says. She also starred in a Broadway production of The Seven Year Itch. City News Service reported that other stage credits included As You Like It, No, No, Nanette and Damn Yankees. Forrest married writer-producer Milo Frank in 1951. He died in 2004. She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney. Career: A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .","Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .\nForrest, whose birth name was Katherine Feeney, had long battled cancer .\nA San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .",
1,"A middle-school teacher in China has inked hundreds of sketches that are beyond be-leaf. Politics teacher Wang Lian, 35, has created 1000 stunning ink drawings covering subjects as varied as cartoon characters and landscapes to animals, birds according toÂ theÂ People's Daily Online. The intricate scribbles on leaves feature Wang's favourite sites across the city of Nanjing, which include the Presidential Palace, Yangtze River Bridge, the ancient Jiming Temple and the Qinhuai River. Natural canvas: Artist and teacher Wang Lian has done hundreds of drawings, like this temple, on leaves she collects in the park and on the streets . Delicate: She uses an ink pen to gently draw the local scenes and buildings on the dried out leaves . 'Although teaching politics is my job, drawing is my passion and hobby,' said Wang. 'I first tried drawing on leaves about 10 years ago and fell in love with it as an art form immediately. 'It's like drawing on very old parchment paper, you have to be really careful that you don't damage the leaf because it is very fragile and this helps focus your attention and abilities.' Wang started giving the drawings away onÂ Christmas Eve in 2012 when her junior high school son came home saying he wanted to prepare some gifts for his classmates. Being an avid painter, Wang decided to give her son's friends unique presents of gingko leaf paintings. Wang loves gingko leaves and will often pick them up along Gingko Avenue, near to her school, in Nanjing in east China's Jiangsu province. Every autumn she collects about 2,000 leaves from the ground to ensure she has enough to cover spoils too. Intricate: Teacher Wang has drawn hundreds of local scenes on leaves she has collected from the park . Hobby: The artist collects leaves every autumn and dries them out so she can sketch these impressive building scenes . 'The colour and shape of gingko leaves are particularly beautiful,' she said. 'I need to collect around 2000 leaves because this will include losses'. She takes them home where she then presses them be

In [15]:
df_train_small = df_train[:10000]
df_test_small = df_test[:2000]
df_val_small = df_val[:5000]

In [17]:
len(df_val_small)

5000

In [18]:
train_params = {
    'max_seq_length': 512,
    'max_length': 128,
    'train_batch_size': 5,
    'eval_batch_size': 5,
    'num_train_epochs': 8,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 10000,
    'use_multiprocessing': False,
    'fp16': False,
    'save_steps': -1,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'no_cache': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'preprocess_inputs': False,
    'num_return_sequences': 1 
}

In [19]:
model = T5Model('t5', 't5-base', args=train_params, use_cuda=cuda)
model.train_model(df_train_small, eval_data=df_val_small)
gc.collect()

  0%|          | 0/10000 [00:00<?, ?it/s]

Using Adafactor for T5


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

Running Epoch 6 of 8:   0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

55265

In [ ]:
%cd /kaggle/working
from IPython.display import FileLinks
FileLinks(r'outputs')

In [ ]:
pred_params = {
        'max_seq_length': 512,
        'use_multiprocessed_decoding': False
        }

model = T5Model('t5', 'outputs/best_model', args=pred_params, use_cuda=cuda) 
pred = model.predict(list(df_test_small['input_text']))

Generating outputs:   0%|          | 0/400 [00:00<?, ?it/s]

In [24]:
random.sample(pred, 5)

['There is no evidence that preoperative hypnosis or physical therapy reduces morbidity or mortality when compared to preoperative hypnosis. These results are based on a small number of studies and therefore the results are not easily generalised to other clinically important and potentially patient-relevant outcomes.',
 'Miniport laparoscopic cholecystectomy can be completed successfully in more than 85% of patients. Patients, in whom elective miniport laparoscopic cholecystectomy was completed successfully, had lower pain than those who underwent standard laparoscopic cholecystectomy. However, because of the lack of information on its safety, miniport laparoscopic cholecystectomy cannot be recommended outside well-designed, randomised clinical trials.',
 'There is no evidence to support the use of beta-blockers for the treatment of dementia. The results of this review are limited by the small number of participants. In addition, there is a need for further well-designed, adequately p

In [25]:
rouge = RougeCalculator(stopwords=True, lang="en")

def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}

In [26]:
rouge_calc(pred, list(df_test_small['target_text']))

{'Rouge_1': 0.11602160899087614,
 'Rouge_2': 0.01802468853296338,
 'Rouge_L': 0.09230411196900178}